In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
plt.style.use('dark_background')

In [2]:
def elastic_pendulum(t,y):
    L, v, theta, w = y
    dLdt = v
    dvdt = (1+L)*w**2-L+np.cos(theta)
    dthetadt = w
    dwdt = -np.sin(theta)/(1+L)-(2/(1+L))*v*w
    return np.array([dLdt,dvdt,dthetadt,dwdt])

In [11]:
elastic_pendulum_sol = solve_ivp(elastic_pendulum,(0,100),y0=[2,0,1.5,0.5],t_eval=np.linspace(0,50,5000))

In [12]:
X = elastic_pendulum_sol.y[0]*np.sin(elastic_pendulum_sol.y[2])
Y = -elastic_pendulum_sol.y[0]*np.cos(elastic_pendulum_sol.y[2])

In [13]:
# taken from the very nice blog post https://scipython.com/blog/the-spring-pendulum/
def plot_spring(L,theta):
    """Plot the spring from (0,0) to (x,y) as the projection of a helix."""
    # Spring turn radius, number of turns
    rs, ns = 0.05, 25
    # Number of data points for the helix
    Ns = 1000
    # We don't draw coils all the way to the end of the pendulum:
    # pad a bit from the anchor and from the bob by these number of points
    ipad1, ipad2 = 100, 150
    w = np.linspace(0, L, Ns)
    # Set up the helix along the x-axis ...
    xp = np.zeros(Ns)
    xp[ipad1:-ipad2] = rs * np.sin(2*np.pi * ns * w[ipad1:-ipad2] / L)
    # ... then rotate it to align with  the pendulum and plot.
    R = np.array([[np.cos(theta), -np.sin(theta)],
                  [np.sin(theta), np.cos(theta)]])
    xs, ys = - R @ np.vstack((xp, w))
    plt.plot(xs, ys, c='w', lw=2)

In [1]:
#!mkdir tmp
#!rm tmp/*.png

In [17]:
for i,t in enumerate(elastic_pendulum_sol.t):
    fig = plt.figure(figsize=(10,10))
    plt.plot(X[:i+1],Y[:i+1],alpha=0.6,c='xkcd:avocado')
    plt.xlim([-3,3])
    plt.ylim([-3,1])
    plot_spring(elastic_pendulum_sol.y[0,i],elastic_pendulum_sol.y[2,i])
    c1 = plt.Circle((X[i], Y[i]), .04, fc='r', ec='r', zorder=10)
    plt.gca().add_patch(c1)
    plt.savefig(f'tmp/img{i:04d}.png',dpi=200)
    plt.close()